In [19]:
ceppo="CCNUC1"
file=f"/home/davide/Downloads/fgenesBresults{ceppo}.html"
#extract from the html the <pre> tag using BeautifulSoup
from bs4 import BeautifulSoup
with open(file, "r", errors='ignore') as f:
    soup = BeautifulSoup(f, "html.parser")
    pre = soup.find("pre")
    #print the content of the <pre> tag as text until first >
    data=pre.text.split("Predicted")[0]
lines=data.splitlines()[8:]
import pandas as pd
# Split each line into fields and convert the result into a DataFrame
df = pd.DataFrame([line.strip().split() for line in lines if line])
#delete columns 4 abd 8
df.drop(columns=[4,8],inplace=True)
df.columns=["Gene number","TU number","Type","NgeneinTU","Strand","CDS","Start","End","Score"]
#df type of start and end are string, convert them to int
df["Start"]=df["Start"].astype(float)
df["End"]=df["End"].astype(float)
operons = df[df['Type'] == 'Op']
# Group by Operon and Strand, then get min Start and max End
operon_ranges = operons.groupby(['TU number', 'Strand']).agg({'Start': 'min', 'End': 'max'}).reset_index()
# Separate positive and negative strand operons
positive_strand_operons = operon_ranges[operon_ranges['Strand'] == '+']
negative_strand_operons = operon_ranges[operon_ranges['Strand'] == '-']
df

,Gene number,TU number,Type,NgeneinTU,Strand,CDS,Start,End,Score
0,1,1,Tu,1,+,CDS,1.0,1038.0,605
1,2,2,Tu,1,+,CDS,1255.0,1581.0,388
2,3,3,Tu,1,+,CDS,1751.0,3358.0,1273
3,4,4,Tu,1,+,CDS,3948.0,4142.0,179
4,5,5,Tu,1,+,CDS,4618.0,5364.0,658
...,...,...,...,...,...,...,...,...,...
5724,5725,3992,Tu,1,+,CDS,6350137.0,6350382.0,198
5725,5726,3993,Tu,1,+,CDS,6350510.0,6350878.0,220
5726,5727,3994,Tu,1,-,CDS,6350935.0,6351450.0,535
5727,5728,3995,Tu,1,-,CDS,6351568.0,6352989.0,1126


In [20]:
gff=f"/home/davide/Desktop/genomiChro/gffs/refseq/{ceppo}.gff"
import pandas as pd


with open(gff, 'r') as file:
    lines = []
    for line in file:
        if line.strip() == "##FASTA":
            break
        if not line.startswith("#"):
            lines.append(line.strip().split('\t'))

df2 = pd.DataFrame(lines)
df2.columns=["seqid","source","type","start","end","score","strand","phase","attributes"]
df2.drop(columns=["source","score","phase"],inplace=True)

#filter only rows where type is CDS
df2=df2[df2["type"]=="CDS"]
df2["start"]=df2["start"].astype(float)
df2["end"]=df2["end"].astype(float)
#df2 delete where seqid is not the one in first row

df2=df2[df2["seqid"]==df2.iloc[0]["seqid"]]
df2['pid'] = df2['attributes'].str.extract('ID=cds-(.*?);')
df2['locus'] = df2['attributes'].str.extract('Parent=gene-(.*?);')
frames = []

for _, row in positive_strand_operons.iterrows():
    df_temp = df2[df2["end"].between(row["Start"], row["End"])]
    
    #pick only the first row of the dataframe but add column operon where put all seqids separated byy |
    df_temp["operonpids"]=df_temp["pid"].str.cat(sep="|")
    df_temp["operon_locus"]=df_temp["locus"].str.cat(sep="|")
    
    #add operon_range column with the start and end of the operon
    df_temp["operon_range"]=str(row["Start"])+"-"+str(row["End"])
    df_temp=df_temp.head(1)
    frames.append(df_temp)

for _, row in negative_strand_operons.iterrows():
    df_temp = df2[df2["start"].between(row["Start"], row["End"])]
    #pick only the last row of the dataframe but add column operon where put all seqids separated byy |
    df_temp["operonpids"]=df_temp["pid"].str.cat(sep="|")
    df_temp["operon_locus"]=df_temp["locus"].str.cat(sep="|")
    df_temp["operon_range"]=str(row["Start"])+"-"+str(row["End"])
    df_temp=df_temp.tail(1)
    frames.append(df_temp)
operons = pd.concat(frames)
frames = []

for _, row in positive_strand_operons.iterrows():
    df_temp = df2[df2["end"].between(row["Start"], row["End"])]
    #pick only the first row of the dataframe but add column operon where put all seqids separated byy |
    df_temp["operon"]=df_temp["seqid"].str.cat(sep="|")
    frames.append(df_temp)

for _, row in negative_strand_operons.iterrows():
    df_temp = df2[df2["start"].between(row["Start"], row["End"])]
    #pick only the last row of the dataframe but add column operon where put all seqids separated byy |
    df_temp["operon"]=df_temp["seqid"].str.cat(sep="|")
    frames.append(df_temp)
operonsall = pd.concat(frames)
df5=df2[~df2["pid"].isin(operonsall["pid"])]
operons=pd.concat([operons,df5])
operons["operonpids"].fillna(operons["pid"],inplace=True)
operons["operon_locus"].fillna(operons["locus"],inplace=True)
locuses=operons["locus"].values.tolist()
operons

/tmp/ipykernel_7843/2914586145.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp["operonpids"]=df_temp["pid"].str.cat(sep="|")
/tmp/ipykernel_7843/2914586145.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp["operon_locus"]=df_temp["locus"].str.cat(sep="|")
/tmp/ipykernel_7843/2914586145.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

,seqid,type,start,end,strand,attributes,pid,locus,operonpids,operon_locus,operon_range
3021,NZ_CP097480.1,CDS,1597160.0,1597600.0,+,ID=cds-WP_250016179.1;Parent=gene-M5J74_RS0746...,WP_250016179.1,M5J74_RS07465,WP_250016179.1|WP_015153510.1,M5J74_RS07465|M5J74_RS07470,1597133.0-1598227.0
296,NZ_CP097480.1,CDS,169657.0,170547.0,+,ID=cds-WP_015152244.1;Parent=gene-M5J74_RS0076...,WP_015152244.1,M5J74_RS00760,WP_015152244.1|WP_015152245.1,M5J74_RS00760|M5J74_RS00765,169762.0-170930.0
3039,NZ_CP097480.1,CDS,1606360.0,1607648.0,+,ID=cds-M5J74_RS33340;Parent=gene-M5J74_RS33340...,M5J74_RS33340,M5J74_RS33340,M5J74_RS33340,M5J74_RS33340,1606360.0-1607648.0
3049,NZ_CP097480.1,CDS,1611870.0,1613477.0,+,ID=cds-WP_250016188.1;Parent=gene-M5J74_RS0754...,WP_250016188.1,M5J74_RS07540,WP_250016188.1|WP_015153524.1,M5J74_RS07540|M5J74_RS07545,1611870.0-1615224.0
3053,NZ_CP097480.1,CDS,1615409.0,1615576.0,+,ID=cds-WP_106217257.1;Parent=gene-M5J74_RS0755...,WP_106217257.1,M5J74_RS07550,WP_106217257.1|WP_250016190.1|WP_041462902.1|W...,M5J74_RS07550|M5J74_RS07555|M5J74_RS07560|M5J7...,1615409.0-1620690.0
...,...,...,...,...,...,...,...,...,...,...,...
11924,NZ_CP097480.1,CDS,6350137.0,6350382.0,+,ID=cds-WP_015157499.1;Parent=gene-M5J74_RS2937...,WP_015157499.1,M5J74_RS29375,WP_015157499.1,M5J74_RS29375,NaN
11926,NZ_CP097480.1,CDS,6350510.0,6350878.0,+,ID=cds-WP_015157500.1;Parent=gene-M5J74_RS2938...,WP_015157500.1,M5J74_RS29380,WP_015157500.1,M5J74_RS29380,NaN
11928,NZ_CP097480.1,CDS,6350935.0,6351450.0,-,ID=cds-WP_106167908.1;Parent=gene-M5J74_RS2938...,WP_106167908.1,M5J74_RS29385,WP_106167908.1,M5J74_RS29385,NaN
11930,NZ_CP097480.1,CDS,6351568.0,6352989.0,-,ID=cds-WP_106217854.1;Parent=gene-M5J74_RS2939...,WP_106217854.1,M5J74_RS29390,WP_106217854.1,M5J74_RS29390,NaN


In [21]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
def sequenze_intergenichetutte(input, output, pids, locuses, max_len=300,c=0,min_len=20):
    """
    Estrae le sequenze intergeniche da un file GenBank e le scrive in un file di output in formato fasta.
    :param input: Il file GenBank di input
    :param output: Il file fasta di output
    :param pids: Il dataframe con i PID dei geni ortologhi, tante colonne quanti i genomi
    :param c: La lunghezza della regione codificante da aggiungere alla sequenza intergenica
    :param max_len: La lunghezza massima della sequenza intergenica
    :param min_len: La lunghezza minima della sequenza intergenica
    :return: None
    """
    intergeniche_plus = []  # Lista per memorizzare le sequenze intergeniche in direzione +
    intergeniche_minus = []  # Lista per memorizzare le sequenze intergeniche in direzione -
    genoma = SeqIO.parse(input, "genbank")
    for sequenze in genoma:
        # sequenze sta per sequenzefold perchè il file genbank potrebbe non contenere un solo cromosoma circolare
        cds_list_plus = []  # Lista per memorizzare le informazioni sui geni in direzione +
        cds_list_minus = []  # Lista per memorizzare le informazioni sui geni in direzione -
        for feature in sequenze.features:
            # Controlla se la feature è un CDS
            if feature.type == "CDS":
                mystart = feature.location.start  # Ottieni l'inizio del CDS
                myend = feature.location.end  # Ottieni la fine del CDS
                mystrand = '-' if feature.location.strand == -1 else '+'
                mystart, myend = int(mystart), int(myend)
                # Check the strand of the CDS and add the info to the appropriate list
                if mystrand == '-' and feature.qualifiers["locus_tag"][0] in locuses:
                    cds_list_minus.append((mystart, myend, feature.qualifiers["locus_tag"][0],
                                           feature.qualifiers["product"][0],
                                           feature.qualifiers.get("gene", [' '])[0],feature.qualifiers.get("protein_id", [' '])[0]))
                elif mystrand == '+' and feature.qualifiers["locus_tag"][0] in locuses:
                    cds_list_plus.append((mystart, myend, feature.qualifiers["locus_tag"][0],
                                          feature.qualifiers["product"][0],
                                          feature.qualifiers.get("gene", [' '])[0],feature.qualifiers.get("protein_id", [' '])[0]))

        
            # Loop attraverso i geni in direzione +
        for i, n in enumerate(cds_list_plus):
            #if n[-1] not in df[col].values:
               # continue
            last_end = 0 if i == 0 else cds_list_plus[i - 1][1] # Ottieni la fine del gene precedente
            this_start = n[0]  # Ottieni l'inizio del gene corrente
            intergene_seq = sequenze.seq[last_end:this_start+c]  # Ottieni la sequenza intergenica
            if len(intergene_seq) <= min_len:  # Se la sequenza intergenica è più corta di min_len, non considerarla e aggiiungi il pid al gene precedente per OPERONE
                if not intergeniche_plus:
                    continue
                # Assuming description and otherpid are defined
                parts = intergeniche_plus[-1].description.split(",")  # Split the description into parts
                pids = parts[0]  # Get the second part (after the comma)
                pids += f"|{operons.loc[operons['locus'] == n[2],'operonpids'].values[0]}"
                parts[0] = pids  # Replace the second part in the list
                locus = parts[1]  # Get the third part (after the comma)
                locus += f"|{operons.loc[operons['locus'] == n[2],'operon_locus'].values[0]}"
                parts[1] = locus  # Replace the third part in the list
                intergeniche_plus[-1].description= ", ".join(parts)
                continue
            pidsoperons=operons.loc[operons['locus'] == n[2],'operonpids'].values[0]
            locusoperons=operons.loc[operons['locus'] == n[2],'operon_locus'].values[0]
            if len(intergene_seq) >= max_len:  # Se la sequenza intergenica è più lunga di max_len, prendi solo le ultime max_len basi(default 300)
                intergene_seq = sequenze.seq[this_start - max_len:this_start+c]
                last_end = this_start - max_len  # Aggiorna quello che sarà l'inizio della sequenza intergenica
            # Aggiungi la sequenza intergenica alla lista come SeqRecord
            description = f"+ pids={pidsoperons}, locus_tags={locusoperons}, pidCCMEE29="
    
            intergeniche_plus.append(
                SeqRecord(
                    intergene_seq,
                    id=n[2],  # id del gene(locus_tag)
                    description=description
                )
            )
        # Loop attraverso i geni in direzione -
        cds_list_minus=cds_list_minus[::-1]
        for i, n in enumerate(cds_list_minus):
            next_start = len(sequenze.seq) if i ==0 else cds_list_minus[i - 1][0]  # Ottieni l'inizio del prossimo gene(gene precedente per la direzione -)
            this_end = n[1]  # Ottieni la fine del gene corrente(inizio del gene per la direzione -)
            intergene_seq = sequenze.seq[this_end-c:next_start]  # Ottieni la sequenza intergenica
            if len(intergene_seq) <= min_len:
                if not intergeniche_minus:
                    continue
                parts = intergeniche_minus[-1].description.split(",")  # Split the description into parts
                pids = parts[0]  # Get the second part (after the comma)
                pids += f"|{operons.loc[operons['locus'] == n[2],'operonpids'].values[0]}"
                parts[0] = pids  # Replace the second part in the list
                locus = parts[1]  # Get the third part (after the comma)
                locus += f"|{operons.loc[operons['locus'] == n[2],'operon_locus'].values[0]}"
                parts[1] = locus  # Replace the third part in the list
                intergeniche_minus[-1].description= ", ".join(parts)
                continue
            pidsoperons=operons.loc[operons['locus'] == n[2],'operonpids'].values[0]
            locusoperons=operons.loc[operons['locus'] == n[2],'operon_locus'].values[0]
            if len(intergene_seq) >= max_len:
                # Se la sequenza intergenica è più lunga di max_len, prendi solo le prime max_len basi(default 300)
                intergene_seq = sequenze.seq[this_end-c:this_end + max_len] if this_end + max_len < len(
                    sequenze.seq) else sequenze.seq[this_end-c:]
            intergene_seq = intergene_seq.reverse_complement()  # Complemento inverso della sequenza intergenica
            description = f"+ pids={pidsoperons}, locus_tags={locusoperons}, pidCCMEE29="
            # Aggiungi la sequenza intergenica alla lista come SeqRecord

            intergeniche_minus.append(
                SeqRecord(
                    intergene_seq,
                    id=n[2],  # id del gene(locus_tag)
                    description=description
                )
            )
    # Unisci le sequenze intergeniche in direzione + e - in una lista
    intergeniche = intergeniche_plus + intergeniche_minus
    # Scrivi le sequenze intergeniche nel file di output
    SeqIO.write(intergeniche, output, "fasta")
from pathlib import Path
input=Path(f"/home/davide/Desktop/genomiChro/annotati_Refseq/{ceppo}.gbff")
output=f"/home/davide/Desktop/genomiChro/provafgenesboperoni/intergeniche_operoni/{ceppo}.fasta"
pidsortologhi=pd.read_pickle("ortologhimmseqs.pkl")
sequenze_intergenichetutte(input, output, pidsortologhi, locuses)

In [151]:
df=pidsortologhi
name=ceppo
col=[col for col in df.columns if name in col and col!="CCMEE_29"][0]
df=df.dropna(subset=[col])
#make a dictionary with the pid of the gene as key and the pidCCMEE_29 of the orthologous gene as value
dictgenes = df.set_index(col)["CCMEE_29"].to_dict()

from Bio import SeqIO
records = list(SeqIO.parse(output, "fasta"))
#change all the description of the records
for record in records:
    pids=record.description.split(",")[0].split("=")[1]
    pids = pids.split('|')
    transformed_pids = [dictgenes[pid] if pid in dictgenes else "No ortologo" for pid in pids]
    transformed_pid_string = '|'.join(transformed_pids)
    record.description=record.description+transformed_pid_string
#write the records in a new file
SeqIO.write(records, f"/home/davide/Desktop/genomiChro/provafgenesboperoni/{ceppo}.fasta", "fasta")


4011

In [106]:
pidsortologhi

,CCMEE_29,result_Chroococcidiopsis_sp._[FACHB-1243]_GCF_014696895.1.faa,result_Chroococcidiopsis_sp._CCNUC1_GCF_023546805.1.faa,result_Chroococcidiopsis_sp._CCMEE_29_GCF_023558375.1.faa,result_Chroococcidiopsis_thermalis_PCC_7203_GCF_000317125.1.faa,result_Chroococcidiopsis_cubana_SAG_39.79_GCF_034438355.1.faa,result_Chroococcidiopsis_sp._CCALA_051_GCF_003015105.1.faa,result_Chroococcidiopsis_sp._TS-821_GCF_002939305.1.faa,result_Chroococcidiopsis_cubana_CCALA_043_GCF_003003835.1.faa,result_Chroococcidiopsis_sp._SAG_2025_GCF_032860985.1.faa
0,WP_009632395.1,WP_039717719.1,WP_039717719.1,WP_009632395.1,WP_039717719.1,WP_039717719.1,WP_039717719.1,WP_104545246.1,WP_039717719.1,WP_039717719.1
1,WP_041548398.1,WP_006196724.1,WP_006196724.1,WP_041548398.1,WP_006196724.1,WP_006196724.1,WP_006196724.1,WP_006276978.1,WP_006196724.1,WP_006196724.1
2,WP_073591590.1,WP_015153091.1,WP_015153091.1,WP_073591590.1,WP_015153091.1,WP_015153091.1,WP_015153091.1,WP_104546135.1,WP_015153091.1,WP_015153091.1
3,WP_190736661.1,WP_199317029.1,NaN,WP_190736661.1,NaN,WP_106168925.1,WP_106218980.1,NaN,WP_106168925.1,WP_317112279.1
4,WP_250121152.1,NaN,NaN,WP_250121152.1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5780,WP_275983394.1,WP_015154006.1,WP_015154006.1,WP_275983394.1,WP_015154006.1,WP_015154006.1,WP_015154006.1,WP_104546721.1,WP_015154006.1,WP_015154006.1
5781,WP_275983395.1,NaN,NaN,WP_275983395.1,NaN,NaN,NaN,NaN,NaN,NaN
5782,WP_275983396.1,WP_192156346.1,WP_250017462.1,WP_275983396.1,WP_015163153.1,WP_199755664.1,WP_106545471.1,WP_104545432.1,WP_199755664.1,WP_039719372.1
5783,WP_275983397.1,NaN,NaN,WP_275983397.1,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
dictgenes['WP_015153795.1']

'WP_250121493.1'

In [10]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
def sequenze_intergenichetutte(input, output, max_len=300,c=0,min_len=20):
    """
    Estrae le sequenze intergeniche da un file GenBank e le scrive in un file di output in formato fasta.
    :param input: Il file GenBank di input
    :param output: Il file fasta di output
    :param pids: Il dataframe con i PID dei geni ortologhi, tante colonne quanti i genomi
    :param c: La lunghezza della regione codificante da aggiungere alla sequenza intergenica
    :param max_len: La lunghezza massima della sequenza intergenica
    :param min_len: La lunghezza minima della sequenza intergenica
    :return: None
    """
    intergeniche_plus = []  # Lista per memorizzare le sequenze intergeniche in direzione +
    intergeniche_minus = []  # Lista per memorizzare le sequenze intergeniche in direzione -
    genoma = SeqIO.parse(input, "genbank")
    for sequenze in genoma:
        # sequenze sta per sequenzefold perchè il file genbank potrebbe non contenere un solo cromosoma circolare
        cds_list_plus = []  # Lista per memorizzare le informazioni sui geni in direzione +
        cds_list_minus = []  # Lista per memorizzare le informazioni sui geni in direzione -
        for feature in sequenze.features:
            # Controlla se la feature è un CDS
            if feature.type == "CDS":
                mystart = feature.location.start  # Ottieni l'inizio del CDS
                myend = feature.location.end  # Ottieni la fine del CDS
                mystrand = '-' if feature.location.strand == -1 else '+'
                mystart, myend = int(mystart), int(myend)
                # Check the strand of the CDS and add the info to the appropriate list
                if mystrand == '-':
                    cds_list_minus.append((mystart, myend, feature.qualifiers["locus_tag"][0],
                                           feature.qualifiers["product"][0],
                                           feature.qualifiers.get("gene", [' '])[0],feature.qualifiers.get("protein_id", [' '])[0]))
                else:
                    cds_list_plus.append((mystart, myend, feature.qualifiers["locus_tag"][0],
                                          feature.qualifiers["product"][0],
                                          feature.qualifiers.get("gene", [' '])[0],feature.qualifiers.get("protein_id", [' '])[0]))

        
            # Loop attraverso i geni in direzione +
        for i, n in enumerate(cds_list_plus):
            #if n[-1] not in df[col].values:
               # continue
            last_end = 0 if i == 0 else cds_list_plus[i - 1][1] # Ottieni la fine del gene precedente
            this_start = n[0]  # Ottieni l'inizio del gene corrente
            intergene_seq = sequenze.seq[last_end:this_start+c]  # Ottieni la sequenza intergenica
            if len(intergene_seq) <= min_len:  # Se la sequenza intergenica è più corta di min_len, non considerarla e aggiiungi il pid al gene precedente per OPERONE
                if not intergeniche_plus:
                    continue
                # Assuming description and otherpid are defined
                parts = intergeniche_plus[-1].description.split(",")  # Split the description into parts
                pids = parts[0]  # Get the second part (after the comma)
                pids += f"|{n[-1]}"
                parts[0] = pids  # Replace the second part in the list
                locus = parts[1]  # Get the third part (after the comma)
                locus += f"|{n[2]}"
                parts[1] = locus  # Replace the third part in the list
                intergeniche_plus[-1].description= ", ".join(parts)
                continue
            if len(intergene_seq) >= max_len:  # Se la sequenza intergenica è più lunga di max_len, prendi solo le ultime max_len basi(default 300)
                intergene_seq = sequenze.seq[this_start - max_len:this_start+c]
                last_end = this_start - max_len  # Aggiorna quello che sarà l'inizio della sequenza intergenica
            # Aggiungi la sequenza intergenica alla lista come SeqRecord
            description = f"+ pids={n[-1]}, locus_tags={n[2]}, pidCCMEE29="
    
            intergeniche_plus.append(
                SeqRecord(
                    intergene_seq,
                    id=n[2],  # id del gene(locus_tag)
                    description=description
                )
            )
        # Loop attraverso i geni in direzione -
        cds_list_minus=cds_list_minus[::-1]
        for i, n in enumerate(cds_list_minus):
            next_start = len(sequenze.seq) if i ==0 else cds_list_minus[i - 1][0]  # Ottieni l'inizio del prossimo gene(gene precedente per la direzione -)
            this_end = n[1]  # Ottieni la fine del gene corrente(inizio del gene per la direzione -)
            intergene_seq = sequenze.seq[this_end-c:next_start]  # Ottieni la sequenza intergenica
            if len(intergene_seq) <= min_len:
                if not intergeniche_minus:
                    continue
                parts = intergeniche_minus[-1].description.split(",")  # Split the description into parts
                pids = parts[0]  # Get the second part (after the comma)
                pids += f"|{n[-1]}"
                parts[0] = pids  # Replace the second part in the list
                locus = parts[1]  # Get the third part (after the comma)
                locus += f"|{n[2]}"
                parts[1] = locus  # Replace the third part in the list
                intergeniche_minus[-1].description= ", ".join(parts)
                continue

            if len(intergene_seq) >= max_len:
                # Se la sequenza intergenica è più lunga di max_len, prendi solo le prime max_len basi(default 300)
                intergene_seq = sequenze.seq[this_end-c:this_end + max_len] if this_end + max_len < len(
                    sequenze.seq) else sequenze.seq[this_end-c:]
            intergene_seq = intergene_seq.reverse_complement()  # Complemento inverso della sequenza intergenica
            description = f"- pids={n[-1]}, locus_tags={n[2]}, pidCCMEE29="
            # Aggiungi la sequenza intergenica alla lista come SeqRecord

            intergeniche_minus.append(
                SeqRecord(
                    intergene_seq,
                    id=n[2],  # id del gene(locus_tag)
                    description=description
                )
            )
    # Unisci le sequenze intergeniche in direzione + e - in una lista
    intergeniche = intergeniche_plus + intergeniche_minus
    # Scrivi le sequenze intergeniche nel file di output
    SeqIO.write(intergeniche, output, "fasta")


In [12]:
from pathlib import Path
dir=Path("/home/davide/Desktop/genomiChro/gffs/refseq")
#itrerate over stem name of dir
for ceppo in [x.stem for x in dir.iterdir()]:
    input=Path(f"/home/davide/Desktop/genomiChro/annotati_Refseq/{ceppo}.gbff")
    output=f"/home/davide/Desktop/genomiChro/provafgenesboperoni/intergeniche_operoni/{ceppo}.fasta"
    df=pd.read_pickle("ortologhimmseqs.pkl")
    sequenze_intergenichetutte(input, output)
    col=[col for col in df.columns if ceppo in col and col!="CCMEE_29"][0]
    df=df.dropna(subset=[col])
    #make a dictionary with the pid of the gene as key and the pidCCMEE_29 of the orthologous gene as value
    dictgenes = df.set_index(col)["CCMEE_29"].to_dict()
    print(ceppo,len(dictgenes))
    from Bio import SeqIO
    records = list(SeqIO.parse(output, "fasta"))
    #change all the description of the records
    for record in records:
        pids=record.description.split(",")[0].split("=")[1]
        pids = pids.split('|')
        transformed_pids = [dictgenes[pid] if pid in dictgenes else "No ortologo" for pid in pids]
        transformed_pid_string = '|'.join(transformed_pids)
        record.description=record.description+transformed_pid_string
    #write the records in a new file
    SeqIO.write(records, output, "fasta")


CCALA_051 3364
CCNUC1 3428
SAG_39.79 3535
FACHB-1243 3354
TS-821 3048
PCC_7203 3385
CCALA_043 3414
SAG_2025 3520
CCMEE_29 5785


In [152]:
    col=[col for col in df.columns if ceppo in col and col!="CCMEE_29"][0]
    df=df.dropna(subset=[col])
    #make a dictionary with the pid of the gene as key and the pidCCMEE_29 of the orthologous gene as value
    dictgenes = df.set_index(col)["CCMEE_29"].to_dict()
    
    from Bio import SeqIO
    records = list(SeqIO.parse(output, "fasta"))
    #change all the description of the records
    for record in records:
        pids=record.description.split(",")[0].split("=")[1]
        pids = pids.split('|')
        transformed_pids = [dictgenes[pid] if pid in dictgenes else "No ortologo" for pid in pids]
        transformed_pid_string = '|'.join(transformed_pids)
        record.description=record.description+transformed_pid_string
    #write the records in a new file
    SeqIO.write(records, f"/home/davide/Desktop/genomiChro/provafgenesboperoni/{ceppo}.fasta", "fasta")

,CCMEE_29,result_Chroococcidiopsis_sp._[FACHB-1243]_GCF_014696895.1.faa,result_Chroococcidiopsis_sp._CCNUC1_GCF_023546805.1.faa,result_Chroococcidiopsis_sp._CCMEE_29_GCF_023558375.1.faa,result_Chroococcidiopsis_thermalis_PCC_7203_GCF_000317125.1.faa,result_Chroococcidiopsis_cubana_SAG_39.79_GCF_034438355.1.faa,result_Chroococcidiopsis_sp._CCALA_051_GCF_003015105.1.faa,result_Chroococcidiopsis_sp._TS-821_GCF_002939305.1.faa,result_Chroococcidiopsis_cubana_CCALA_043_GCF_003003835.1.faa,result_Chroococcidiopsis_sp._SAG_2025_GCF_032860985.1.faa
0,WP_009632395.1,WP_039717719.1,WP_039717719.1,WP_009632395.1,WP_039717719.1,WP_039717719.1,WP_039717719.1,WP_104545246.1,WP_039717719.1,WP_039717719.1
1,WP_041548398.1,WP_006196724.1,WP_006196724.1,WP_041548398.1,WP_006196724.1,WP_006196724.1,WP_006196724.1,WP_006276978.1,WP_006196724.1,WP_006196724.1
2,WP_073591590.1,WP_015153091.1,WP_015153091.1,WP_073591590.1,WP_015153091.1,WP_015153091.1,WP_015153091.1,WP_104546135.1,WP_015153091.1,WP_015153091.1
5,WP_250121153.1,WP_192153441.1,WP_250015281.1,WP_250121153.1,WP_015152618.1,WP_158631858.1,WP_158265939.1,WP_104546600.1,WP_106170952.1,WP_317105904.1
6,WP_250121154.1,WP_192155592.1,WP_015153906.1,WP_250121154.1,WP_015153906.1,WP_015153906.1,WP_015153906.1,WP_104546487.1,WP_015153906.1,WP_317107024.1
...,...,...,...,...,...,...,...,...,...,...
5772,WP_275983386.1,WP_192160381.1,WP_106169263.1,WP_275983386.1,WP_015156394.1,WP_106169263.1,WP_106169263.1,WP_104548602.1,WP_106169263.1,WP_317108959.1
5775,WP_275983389.1,WP_192161005.1,WP_041463362.1,WP_275983389.1,WP_041463362.1,WP_041463362.1,WP_041463362.1,WP_104548644.1,WP_041463362.1,WP_041463362.1
5776,WP_275983390.1,WP_192161007.1,WP_015156154.1,WP_275983390.1,WP_015156154.1,WP_015156154.1,WP_106547151.1,WP_104548646.1,WP_015156154.1,WP_317107872.1
5780,WP_275983394.1,WP_015154006.1,WP_015154006.1,WP_275983394.1,WP_015154006.1,WP_015154006.1,WP_015154006.1,WP_104546721.1,WP_015154006.1,WP_015154006.1


In [2]:
import pandas as pd
pidsortologhi=pd.read_pickle("ortologhimmseqs.pkl")
pidsortologhi

,CCMEE_29,result_Chroococcidiopsis_sp._[FACHB-1243]_GCF_014696895.1.faa,result_Chroococcidiopsis_sp._CCNUC1_GCF_023546805.1.faa,result_Chroococcidiopsis_sp._CCMEE_29_GCF_023558375.1.faa,result_Chroococcidiopsis_thermalis_PCC_7203_GCF_000317125.1.faa,result_Chroococcidiopsis_cubana_SAG_39.79_GCF_034438355.1.faa,result_Chroococcidiopsis_sp._CCALA_051_GCF_003015105.1.faa,result_Chroococcidiopsis_sp._TS-821_GCF_002939305.1.faa,result_Chroococcidiopsis_cubana_CCALA_043_GCF_003003835.1.faa,result_Chroococcidiopsis_sp._SAG_2025_GCF_032860985.1.faa
0,WP_009632395.1,WP_039717719.1,WP_039717719.1,WP_009632395.1,WP_039717719.1,WP_039717719.1,WP_039717719.1,WP_104545246.1,WP_039717719.1,WP_039717719.1
1,WP_041548398.1,WP_006196724.1,WP_006196724.1,WP_041548398.1,WP_006196724.1,WP_006196724.1,WP_006196724.1,WP_006276978.1,WP_006196724.1,WP_006196724.1
2,WP_073591590.1,WP_015153091.1,WP_015153091.1,WP_073591590.1,WP_015153091.1,WP_015153091.1,WP_015153091.1,WP_104546135.1,WP_015153091.1,WP_015153091.1
3,WP_190736661.1,WP_199317029.1,NaN,WP_190736661.1,NaN,WP_106168925.1,WP_106218980.1,NaN,WP_106168925.1,WP_317112279.1
4,WP_250121152.1,NaN,NaN,WP_250121152.1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5780,WP_275983394.1,WP_015154006.1,WP_015154006.1,WP_275983394.1,WP_015154006.1,WP_015154006.1,WP_015154006.1,WP_104546721.1,WP_015154006.1,WP_015154006.1
5781,WP_275983395.1,NaN,NaN,WP_275983395.1,NaN,NaN,NaN,NaN,NaN,NaN
5782,WP_275983396.1,WP_192156346.1,WP_250017462.1,WP_275983396.1,WP_015163153.1,WP_199755664.1,WP_106545471.1,WP_104545432.1,WP_199755664.1,WP_039719372.1
5783,WP_275983397.1,NaN,NaN,WP_275983397.1,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
ceppo="CCMEE_29"
df=pidsortologhi
col=[col for col in df.columns if ceppo in col and col!="CCMEE_29"][0]
df=df.dropna(subset=[col])
#make a dictionary with the pid of the gene as key and the pidCCMEE_29 of the orthologous gene as value
dictgenes = df.set_index(col)["CCMEE_29"].to_dict()
dictgenes

{'WP_009632395.1': 'WP_009632395.1',
 'WP_041548398.1': 'WP_041548398.1',
 'WP_073591590.1': 'WP_073591590.1',
 'WP_190736661.1': 'WP_190736661.1',
 'WP_250121152.1': 'WP_250121152.1',
 'WP_250121153.1': 'WP_250121153.1',
 'WP_250121154.1': 'WP_250121154.1',
 'WP_250121155.1': 'WP_250121155.1',
 'WP_250121156.1': 'WP_250121156.1',
 'WP_250121157.1': 'WP_250121157.1',
 'WP_250121158.1': 'WP_250121158.1',
 'WP_250121159.1': 'WP_250121159.1',
 'WP_250121160.1': 'WP_250121160.1',
 'WP_250121161.1': 'WP_250121161.1',
 'WP_250121162.1': 'WP_250121162.1',
 'WP_250121163.1': 'WP_250121163.1',
 'WP_250121164.1': 'WP_250121164.1',
 'WP_250121165.1': 'WP_250121165.1',
 'WP_250121166.1': 'WP_250121166.1',
 'WP_250121167.1': 'WP_250121167.1',
 'WP_250121168.1': 'WP_250121168.1',
 'WP_250121169.1': 'WP_250121169.1',
 'WP_250121170.1': 'WP_250121170.1',
 'WP_250121171.1': 'WP_250121171.1',
 'WP_250121172.1': 'WP_250121172.1',
 'WP_250121173.1': 'WP_250121173.1',
 'WP_250121174.1': 'WP_250121174.1',
 

In [4]:
len(dictgenes)

5785

In [23]:
for ceppo in ["PCC_7203","CCMEE_29","CCNUC1"]:
    output=f"/home/davide/Desktop/genomiChro/provafgenesboperoni/intergeniche_operoni/{ceppo}.fasta"
    df=pd.read_pickle("ortologhimmseqs.pkl")
    col=[col for col in df.columns if ceppo in col and col!="CCMEE_29"][0]
    df=df.dropna(subset=[col])
    #make a dictionary with the pid of the gene as key and the pidCCMEE_29 of the orthologous gene as value
    dictgenes = df.set_index(col)["CCMEE_29"].to_dict()
    print(ceppo,len(dictgenes))
    from Bio import SeqIO
    records = list(SeqIO.parse(output, "fasta"))
    #change all the description of the records
    for record in records:
        pids=record.description.split(",")[0].split("=")[1]
        pids = pids.split('|')
        transformed_pids = [dictgenes[pid] if pid in dictgenes else "No ortologo" for pid in pids]
        transformed_pid_string = '|'.join(transformed_pids)
        record.description=record.description+transformed_pid_string
    #write the records in a new file
    SeqIO.write(records, output, "fasta")

PCC_7203 3385
CCMEE_29 5785
CCNUC1 3428


In [25]:
pidsortologhi=pd.read_pickle("ortologhimmseqs.pkl")
#list of the pids fo column CCMEE_29
pids_CCMEE_29=pidsortologhi["CCMEE_29"].values.tolist()
from Bio import SeqIO
from pathlib import Path

# Initialize an empty dictionary
pid_dict = {}

# Directory containing the fasta files
fasta_dir = Path("/home/davide/Desktop/genomiChro/provafgenesboperoni/intergeniche_operoni")

# Iterate over each pid
for pid in pids_CCMEE_29:
    # Iterate over all the fasta files in the directory
    for fasta_file in fasta_dir.iterdir():
        if fasta_file.stem == "CCMEE_29":
            continue
        # Parse the fasta file
        records = SeqIO.parse(str(fasta_file), "fasta")
        # Iterate over each record
        for record in records:
            # Check if the pid is in the description of the record
            if pid in record.description:
                # Add it to the dictionary
                pid_dict.setdefault(pid, {})[fasta_file.name] = str(record.seq)

print(pid_dict)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [1]:
 # open with open("piddict.pkl", 
import pickle
with open("piddict.pkl", "rb") as f:
    pid_dict = pickle.load(f)
pid_dict

{'WP_009632395.1': {'TS-821.fasta': 'ATCAGGAAATCTTTTGCCCAAAAACCGCGTACTATTAGTTTAATGTATTATTTGTTACTAAATACTGAGATAGTGGGATGCTATTTAAAAATTCATGTTTCACTCACAAGACAAACATTGCACCTGTCATCCTTTTTATGGAAAATAACGAATTGACGGAATGTGACATCATCTACTAAATTAACTTTGTTTTAGAACTGATGTGTTAGTAGCTTTTATTTGAATGTTCAGCAAATTTAAGTAAGGAGTGAAGAAAA',
  'SAG_39.79.fasta': 'GCGATCGCACTGACTGCCTTCTTCTAAAAAATGTATTAATTAATACGATACTTCACACTAGAGGATGATACTACAAATAAGTTCCAGCTATTGCATTTGGGATCGATCGAGTGTAGAAAAACAGGATGGAGGCAATCTACGCAGTAATTCAAACAGTGCGATCGCACTCAAACTAAGAGATAATTTTCCTTTGGTGAATCACAGCAGTAAAACACACCAATATAAAACGATAAACAAACGGGAGTGAGAAAA',
  'FACHB-1243.fasta': 'GCGATCGCACTGACTGCCTTCTTCTAAAAAATGTATTAATCAATACGATACTTCACAAGAGGGAATGATACTACAAATAAGTTCCAGCTATTGCATTTGGGATCGATCGAGTGTAGAAAAACAGGATGGAGGCAATCTACGCAGTAATTCAAACAGTGCGATCGCACTCAAACTAAGAGATAATTTTCATTTGGTGAATCACAGCAGTAAAACACACCAATATAAAACGATAAACAAACGGGAGTGAGAAAA',
  'CCALA_043.fasta': 'GCGATCGCACTGACTGCCTTCTTCTAAAAAATGTATTAATTAATACGATACTTCACACTAGAGGATGATACTACAAATAAGTTCCAGCTATTGCATTTGGGATCGATCGAGTGTAGAAAAACAGGA

In [12]:
file="/home/davide/Desktop/genomiChro/annotati_Refseq/CCMEE_29.gbff"
from Bio import SeqIO
genoma = SeqIO.parse(file, "genbank")
#find protein_id of the gene named lexA
for sequenze in genoma:
    for feature in sequenze.features:
        if feature.type == "CDS" and feature.qualifiers.get("gene", [' '])[0] == "recA":
            print(feature.qualifiers["protein_id"])


['WP_250121493.1']


In [4]:
pid_dict['WP_250121220.1'].values()

dict_values(['GCAGCGAGTAATATTAAACTCAGTTACCTATTACCGACTACCAAATCATCATCTCAACAAAAAATTACTTTATTTGTAGTTTTTTTTGCTATTCGGGGCAATAATGTAGTACGAATGTTCTTATAGACCTTTGTATTTTGGCTTCCGAATT', 'TGAGTGGTGCGTGACAAATCACCAATTACCAATCACCAATTACCAATCACCAATTACCAACTACCAACTACCAACTACCAAACAAAAACTTTTTTTTGCCATTAGGGGCAATTTTGTAGTACGACTGTTCTAAGAGGTCTATATCGCGATTGAAACACACTAAACT', 'AAACTTTTTTTTGCCATTAGGGGCAATTTTGTAGTACGACTGTTCTAAGAGGTCTATATCGCGATTGAAACACACTAAACT', 'TGAGTGGTGCGTGACAAATCACCAATTACCAATCACCAATTACCAATCACCAATTACCAACTACCAACTACCAACTACCAAACAAAAACTTTTTTTTGCCATTAGGGGCAATTTTGTAGTACGACTGTTCTAAGAGGTCTATATCGCGATTGAAACACACTAAACT', 'TGAGTCGGGAGTCGGGATAAGAGAGCTGGGGGAGCAATTACCAATTACCAATTACCAAATAAAAACTTTTTTTTGCCATTAGGGGCAATTTTGTAGTACGACTGTTCTAAGAGGTCTATATCGCGATTGAAACACACTAAACT', 'TGAGTGGTGCGTGACAAATCACCAATTACCAACTACCAATTACTAACTACCAACTACCAACTACCAAACAAAAACTTTTTTTTGCCATTAGGGGCAATTTTGTAGTACGACTGTTCTAAGAGGTCTATATCGCGATTGAAACACACTAAACT', 'TGAGTGGTGCGTGACAAATCACCAATTACCAATCACCAATTACCAATTACTAACTACCAACTACCAACTACCAACTACCAAACAAAAACTTTTTTTTGCCATT

In [7]:
#find motif AGT.{3,10}}ACT in the sequences of the genes with pid WP_250121220.1 pid_dict['WP_250121220.1'].values()
import re
motif = re.compile("AGT.{5,12}CTA")
for seq in pid_dict['WP_250121220.1'].values():
    match = motif.search(seq)
    if match:
        print(match.group())

AGTTTTTTTTGCTA
AGTACGACTGTTCTA
AGTACGACTGTTCTA
AGTACGACTGTTCTA
AGTACGACTGTTCTA
AGTACGACTGTTCTA
AGTACGACTGTTCTA
AGTACGACTGTTCTA


In [9]:
with open("/home/davide/PycharmProjects/TESI2/piddict.pkl", "rb") as f:
    diz = pickle.load(f)
#print first key value pair
print(list(diz.items())[0])

('WP_009632395.1', {'TS-821.fasta': 'ATCAGGAAATCTTTTGCCCAAAAACCGCGTACTATTAGTTTAATGTATTATTTGTTACTAAATACTGAGATAGTGGGATGCTATTTAAAAATTCATGTTTCACTCACAAGACAAACATTGCACCTGTCATCCTTTTTATGGAAAATAACGAATTGACGGAATGTGACATCATCTACTAAATTAACTTTGTTTTAGAACTGATGTGTTAGTAGCTTTTATTTGAATGTTCAGCAAATTTAAGTAAGGAGTGAAGAAAA', 'SAG_39.79.fasta': 'GCGATCGCACTGACTGCCTTCTTCTAAAAAATGTATTAATTAATACGATACTTCACACTAGAGGATGATACTACAAATAAGTTCCAGCTATTGCATTTGGGATCGATCGAGTGTAGAAAAACAGGATGGAGGCAATCTACGCAGTAATTCAAACAGTGCGATCGCACTCAAACTAAGAGATAATTTTCCTTTGGTGAATCACAGCAGTAAAACACACCAATATAAAACGATAAACAAACGGGAGTGAGAAAA', 'FACHB-1243.fasta': 'GCGATCGCACTGACTGCCTTCTTCTAAAAAATGTATTAATCAATACGATACTTCACAAGAGGGAATGATACTACAAATAAGTTCCAGCTATTGCATTTGGGATCGATCGAGTGTAGAAAAACAGGATGGAGGCAATCTACGCAGTAATTCAAACAGTGCGATCGCACTCAAACTAAGAGATAATTTTCATTTGGTGAATCACAGCAGTAAAACACACCAATATAAAACGATAAACAAACGGGAGTGAGAAAA', 'CCALA_043.fasta': 'GCGATCGCACTGACTGCCTTCTTCTAAAAAATGTATTAATTAATACGATACTTCACACTAGAGGATGATACTACAAATAAGTTCCAGCTATTGCATTTGGGATCGATCGAGTGTAGAAAAACAGGATGGAGG

In [10]:
with open("/home/davide/PycharmProjects/TESI2/dizop.pickle", "rb") as f:
    diz = pickle.load(f)
#print first key value pair
print(list(diz.items())[0])

('WP_250125946.1', {'TS-821': ['CACAACGTGTCCTCTGGTAAATACTTGTAGCAACTAGCGTTCCAACATTGAGCAGAATCCCTATGTGAAAGCATTTAATTGCGGCTAACACTGCTTACAAGTTAGGTAGTCATAGGGTATTCTTTAGCAAAAGAGAAAAAGACAACTGTATATCATCTGAAACATCAAAGGAATTAATAAAAAATGCGGCATAGTTTGCGCCTTCGAGAAAATATATTTTACTCAACCATTTCAAATACTATAAGTAGTAATAAAAATCGCTAAGGAAAATATCTATTTAAGATCTTAGGTTTAACTTTT'], 'SAG_39.79': ['TGGCAACTACGTTCGGTCTGAACTGTGGAAACTACCAACCAACTGATAAAATTAACTGCGAATTTAGTTAATTTAAGGGTAGATGTTAAAAATAGGTGTTGCGCTTAAGCTAACCTGGTAAAGGTTTAGCAAGCATGGTTTAACCTTTAAGACGAGAGGCTCACTAAAACTTAGGAAGGGTGAATGTAAAATCGATTGAGATCCATAACATACAATTCATAAAATGATGCATAAAAAATTGTTATAATTGAGAATTCGACGTTAAATAGTTTTTGGAAAATTTCTAGGTATTAAATTTCT'], 'FACHB-1243': ['TGACAACTACGTTCGGTCTGAACTGTGGAAACTACCAACCAACTGATAAAATTAACTGCGAATTTAGTTAATTTAAAGGTAGATGTTAAAAATAGGTGTTGCGCTTAAGCTAACCTGGTAAAGGTTTAGCAAGCATGGTTTAACTTCCCAGACGAGATTCTCACTAAAACTTAGGAAGGATAAATGTAAAATCGATTGAGATCCATAGCATACAATTCATAAAATGATGCATAAAAAATTGTTATAATTGAGAATTCGACGTTAAATAGTTTTTGGAAAATTTTTAGGTATTAAATTTCT'], 'CCALA_043': ['TGGCAACTAC

In [15]:
import pandas as pd
operoniscore=pd.read_csv('/home/davide/PycharmProjects/TESI2/scoresfgenespal.txt',sep='\t',header=None,skiprows=1)
operoniscore.columns=["sequence_name","operone","lunghezza_intergenica","score","score_refined","motivo","posizione","info_geni"]
operoniscore["protein_id"]=operoniscore["operone"].str.split("|")
operoniscore=operoniscore.explode("protein_id")
df_ann=pd.read_csv("/home/davide/Desktop/CCMEE29Annotazioni.csv")

df=pd.merge(operoniscore,df_ann,on="protein_id")
# #sort the dataframe by the column score_refined
df=df.sort_values(by="score_refined",ascending=False)
# #rename sequence_name column
#drop
df=df.rename(columns={"sequence_name":"locus_tag_iniziooperone"})
#save as excel

df

,locus_tag_iniziooperone,operone,lunghezza_intergenica,score,score_refined,motivo,posizione,info_geni,protein_id,GOs_interpro,GOs_eggnog,product,gene,GO_process,GO_component,GO_function,GO_process_pannzer2,KEGG_brite,pathways
1446,LAU37_RS16010,WP_250121493.1,300,10.52,21.05,TAGTATATCTGCACTA,-67,Gene WP_250121493.1. Score ortologhi= TS-821: ...,WP_250121493.1,GO:0003697(InterPro)|GO:0005524(InterPro)|GO:0...,"GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",recombinase RecA,recA,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","['GO:0009432 - SOS response', 'GO:0006310 - DN...",ko03400 DNA repair and recombination proteins,Homologous recombination
1312,LAU37_RS14470,WP_250121220.1|WP_250121221.1,128,10.21,20.21,TAGTACGAATGTTCTA,-48,Gene WP_250121220.1. Score ortologhi= TS-821: ...,WP_250121220.1,-,"GO:0000976,GO:0001067,GO:0001130,GO:0001216,GO...",transcriptional repressor LexA,lexA,['GO:0006282 - regulation of DNA repair [Evide...,NaN,['GO:0004252 - serine-type endopeptidase activ...,NaN,"ko01002 Peptidases and inhibitors, ko03400 DNA...",NaN
1313,LAU37_RS14470,WP_250121220.1|WP_250121221.1,128,10.21,20.21,TAGTACGAATGTTCTA,-48,Gene WP_250121220.1. Score ortologhi= TS-821: ...,WP_250121221.1,GO:0003677(InterPro)|GO:0005524(InterPro)|GO:0...,-,DNA phosphorothioation system restriction enzyme,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1330,LAU37_RS14595,WP_250121243.1|WP_250121244.1|WP_250121245.1|W...,300,10.43,19.30,TAGTACATATGTACTA,-208,Gene WP_250121243.1. Score ortologhi= SAG_39.7...,WP_250121247.1,GO:0016114(InterPro)|GO:0050515(InterPro),"GO:0003674,GO:0003824,GO:0006793,GO:0006796,GO...",4-(cytidine 5'-diphospho)-2-C-methyl-D-erythri...,ispE,['GO:0016114 - terpenoid biosynthetic process ...,NaN,"[""GO:0050515 - 4-(cytidine 5'-diphospho)-2-C-m...",NaN,NaN,"Metabolic pathways, Biosynthesis of secondary ..."
1329,LAU37_RS14595,WP_250121243.1|WP_250121244.1|WP_250121245.1|W...,300,10.43,19.30,TAGTACATATGTACTA,-208,Gene WP_250121243.1. Score ortologhi= SAG_39.7...,WP_250121246.1,GO:0000154(InterPro)|GO:0000179(InterPro),"GO:0000154,GO:0000179,GO:0001510,GO:0003674,GO...",16S rRNA (adenine(1518)-N(6)/adenine(1519)-N(6...,rsmA,NaN,NaN,"['GO:0000179 - rRNA (adenine-N6,N6-)-dimethylt...",NaN,ko03009 Ribosome biogenesis,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4076,LAU37_RS07855,WP_250125029.1,24,-6.74,-6.85,TTCAACAAAGGCAAAA,-18,Gene WP_250125029.1. Score ortologhi= TS-821: ...,WP_250125029.1,-,-,class I SAM-dependent methyltransferase,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3186,LAU37_RS18505,WP_250121957.1,23,-7.02,-7.02,AGGGGTTAGGGTTAGA,-16,NaN,WP_250121957.1,NaN,NaN,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1047,LAU37_RS11355,WP_250125671.1,39,-6.30,-7.27,CAAAATCCAAAATCTA,-21,Gene WP_250125671.1. Score ortologhi= TS-821: ...,WP_250125671.1,-,-,phosphate ABC transporter permease,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259,LAU37_RS17575,WP_250121784.1|WP_250121785.1,24,-7.28,-7.36,CTTTATATATCTCTGA,-17,Gene WP_250121784.1. Score ortologhi= TS-821: ...,WP_250121784.1,GO:0016020(InterPro),"GO:0003674,GO:0005215,GO:0005575,GO:0005623,GO...",twin-arginine translocase subunit TatC,tatC,['GO:0015628 - protein secretion by the type I...,['GO:0005886 - plasma membrane [Evidence IEA]'],NaN,GO:0043953 - protein transport by the Tat complex,ko02044 Secretion system,"Protein export, Bacterial secretion system"


In [16]:
import polars as pl
dfpvalue=pl.read_csv("provastats.txt",separator="\t",has_header=False)
#create numpy arrays from the columns
import numpy as np
array1=dfpvalue["column_1"].to_numpy()
scores=dfpvalue["column_2"].to_numpy()
scoresrefined=dfpvalue["column_3"].to_numpy()
def calculate_pvalue(score):
    n=len(scores)
    pvalues=[]
    pvalues.append(np.sum(scores>score)/n)
    x=np.sum(scores>score)/n
    return x
def calculate_pvalue_refined(score_refined):
    n=len(scores)
    pvalues=[]
    pvalues.append(np.sum(scoresrefined>score_refined)/n)
    x=np.sum(scoresrefined>score_refined)/n
    return x
#now apply the function to the dataframe to create two new columns pvalue and pvalue_refined
df["pvalue"]=df["score"].apply(calculate_pvalue)
df["pvalue_refined"]=df["score_refined"].apply(calculate_pvalue_refined)
df

,locus_tag_iniziooperone,operone,lunghezza_intergenica,score,score_refined,motivo,posizione,info_geni,protein_id,GOs_interpro,...,product,gene,GO_process,GO_component,GO_function,GO_process_pannzer2,KEGG_brite,pathways,pvalue,pvalue_refined
1446,LAU37_RS16010,WP_250121493.1,300,10.52,21.05,TAGTATATCTGCACTA,-67,Gene WP_250121493.1. Score ortologhi= TS-821: ...,WP_250121493.1,GO:0003697(InterPro)|GO:0005524(InterPro)|GO:0...,...,recombinase RecA,recA,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","['GO:0009432 - SOS response', 'GO:0006310 - DN...",ko03400 DNA repair and recombination proteins,Homologous recombination,0.000000,0.000000
1312,LAU37_RS14470,WP_250121220.1|WP_250121221.1,128,10.21,20.21,TAGTACGAATGTTCTA,-48,Gene WP_250121220.1. Score ortologhi= TS-821: ...,WP_250121220.1,-,...,transcriptional repressor LexA,lexA,['GO:0006282 - regulation of DNA repair [Evide...,NaN,['GO:0004252 - serine-type endopeptidase activ...,NaN,"ko01002 Peptidases and inhibitors, ko03400 DNA...",NaN,0.000000,0.000000
1313,LAU37_RS14470,WP_250121220.1|WP_250121221.1,128,10.21,20.21,TAGTACGAATGTTCTA,-48,Gene WP_250121220.1. Score ortologhi= TS-821: ...,WP_250121221.1,GO:0003677(InterPro)|GO:0005524(InterPro)|GO:0...,...,DNA phosphorothioation system restriction enzyme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
1330,LAU37_RS14595,WP_250121243.1|WP_250121244.1|WP_250121245.1|W...,300,10.43,19.30,TAGTACATATGTACTA,-208,Gene WP_250121243.1. Score ortologhi= SAG_39.7...,WP_250121247.1,GO:0016114(InterPro)|GO:0050515(InterPro),...,4-(cytidine 5'-diphospho)-2-C-methyl-D-erythri...,ispE,['GO:0016114 - terpenoid biosynthetic process ...,NaN,"[""GO:0050515 - 4-(cytidine 5'-diphospho)-2-C-m...",NaN,NaN,"Metabolic pathways, Biosynthesis of secondary ...",0.000000,0.000000
1329,LAU37_RS14595,WP_250121243.1|WP_250121244.1|WP_250121245.1|W...,300,10.43,19.30,TAGTACATATGTACTA,-208,Gene WP_250121243.1. Score ortologhi= SAG_39.7...,WP_250121246.1,GO:0000154(InterPro)|GO:0000179(InterPro),...,16S rRNA (adenine(1518)-N(6)/adenine(1519)-N(6...,rsmA,NaN,NaN,"['GO:0000179 - rRNA (adenine-N6,N6-)-dimethylt...",NaN,ko03009 Ribosome biogenesis,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4076,LAU37_RS07855,WP_250125029.1,24,-6.74,-6.85,TTCAACAAAGGCAAAA,-18,Gene WP_250125029.1. Score ortologhi= TS-821: ...,WP_250125029.1,-,...,class I SAM-dependent methyltransferase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.995575,0.997269
3186,LAU37_RS18505,WP_250121957.1,23,-7.02,-7.02,AGGGGTTAGGGTTAGA,-16,NaN,WP_250121957.1,NaN,...,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.996629,0.997688
1047,LAU37_RS11355,WP_250125671.1,39,-6.30,-7.27,CAAAATCCAAAATCTA,-21,Gene WP_250125671.1. Score ortologhi= TS-821: ...,WP_250125671.1,-,...,phosphate ABC transporter permease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.993292,0.998273
3259,LAU37_RS17575,WP_250121784.1|WP_250121785.1,24,-7.28,-7.36,CTTTATATATCTCTGA,-17,Gene WP_250121784.1. Score ortologhi= TS-821: ...,WP_250121784.1,GO:0016020(InterPro),...,twin-arginine translocase subunit TatC,tatC,['GO:0015628 - protein secretion by the type I...,['GO:0005886 - plasma membrane [Evidence IEA]'],NaN,GO:0043953 - protein transport by the Tat complex,ko02044 Secretion system,"Protein export, Bacterial secretion system",0.997408,0.998433


In [15]:
from goatools.obo_parser import GODag
obodag = GODag("go-basic.obo")

go-basic.obo: fmt(1.2) rel(2024-04-24) 45,667 Terms


In [18]:
df.to_csv("/home/davide/Desktop/operoniscorestream.csv")

In [3]:
from goatools.base import get_godag
godag = get_godag('go-basic.obo', optional_attrs='relationship')

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2024-04-24) 45,667 Terms; optional_attrs(relationship)


In [4]:
GO_ID="GO:0009432"
from goatools.gosubdag.gosubdag import GoSubDag
gosubdag_r0 = GoSubDag([GO_ID], godag, prt=None)
print('{GO} ancestors: {P}'.format(
    GO=GO_ID,
    P=gosubdag_r0.rcntobj.go2ancestors[GO_ID]))

GO:0009432 ancestors: {'GO:0033554', 'GO:0051716', 'GO:0008150', 'GO:0009987', 'GO:0006974', 'GO:0050896', 'GO:0006950'}


In [10]:
GO_ID="GO:0043066"
term= godag[GO_ID]
ancestor_terms = term.get_all_parents()
print(ancestor_terms)
print(term.get_all_upper())

{'GO:0050794', 'GO:0008150', 'GO:0048523', 'GO:0043069', 'GO:0043067', 'GO:0042981', 'GO:0050789', 'GO:0065007', 'GO:0048519'}
{'GO:0008219', 'GO:0043067', 'GO:0012501', 'GO:0006915', 'GO:0048519', 'GO:0050794', 'GO:0008150', 'GO:0009987', 'GO:0048523', 'GO:0043069', 'GO:0042981', 'GO:0050789', 'GO:0065007'}


In [23]:
df=pd.read_csv("/home/davide/Desktop/operoniscorestream.csv")

In [22]:
import re
def extract_GO_terms(row):
    row_str = ' '.join(row.astype(str))
    go_terms = re.findall(r'GO:\d+', row_str)
    unique_go_terms = list(set(go_terms))
    return unique_go_terms

df['GOs'] = df.apply(extract_GO_terms, axis=1)
def extract_ancestors(go_id):
    """ Function to get all ancestor terms for a given GO term """
    if go_id not in godag:
        return []
    term = godag[go_id]
    ancestors = term.get_all_upper()
    return [go_id] + list(ancestors)
GOs_background=df["GOs"].tolist()
print(len(GOs_background))
GOs_background=[item for sublist in GOs_background for item in sublist]
GOs_background=[item for sublist in (extract_ancestors(x) for x in GOs_background) for item in sublist]
print(len(GOs_background))
newGOs_background=[]
for x in GOs_background:
    GO=godag[x].namespace + " - "+godag[x].name+" L"+str(godag[x].level)+" D"+str(godag[x].depth)
    newGOs_background.append(GO)
#filter newGOs_background to remove string that dont start with biological_process
newGOs_background=[x for x in newGOs_background if x.startswith("biological_process")]
#print(newGOs_background)
print(len(newGOs_background))

4844
131947
93927


In [25]:
import re
import pandas as pd
def extract_GO_terms(row):
    row_str = ' '.join(row.astype(str))
    go_terms = re.findall(r'GO:\d+', row_str)
    unique_go_terms = list(set(go_terms))
    return unique_go_terms

df['GOs'] = df.apply(extract_GO_terms, axis=1)
def extract_ancestors(go_id):
    """ Function to get all ancestor terms for a given GO term """
    if go_id not in godag:
        return []
    term = godag[go_id]
    ancestors = term.get_all_upper()
    return [go_id] + list(ancestors)
GOs_background=df["GOs"].tolist()

GOs_background=[item for sublist in GOs_background for item in sublist]
GOs_background=[item for sublist in (extract_ancestors(x) for x in GOs_background) for item in sublist]

newGOs_background=[]
for x in GOs_background:
    GO=godag[x].namespace + " - "+godag[x].name+" L"+str(godag[x].level)+" D"+str(godag[x].depth)
    newGOs_background.append(GO)
#filter newGOs_background to remove string that dont start with biological_process
newGOs_background=[x for x in newGOs_background if x.startswith("biological_process")]
from collections import Counter
from scipy.stats import hypergeom
from statsmodels.stats.multitest import multipletests

def GO_enrichments(pvalue,refined=True,l=0,d=0):
    background=newGOs_background
    #filter newGOs_background to remove elements with level and depth less than l and d
    if l!=0:
        background=[x for x in background if int(x.split(" ")[-2][1:])>=l]
    if d!=0:
        background=[x for x in background if int(x.split(" ")[-1][1:])>=d]
    #delete the ones with positive regulation or negative regulation
    background=[x for x in background if "regulation" not in x]
    col="pvalue_refined" if refined else "pvalue"
    df_cut=df[df[col]<pvalue]
    GOs=df_cut["GOs"]
    GOs=[item for sublist in GOs for item in sublist]
    GOs = [item for sublist in (extract_ancestors(x) for x in GOs) for item in sublist]
    newGOs=[]
    for x in GOs:
        GO=godag[x].namespace + " - "+godag[x].name+" L"+str(godag[x].level)+" D"+str(godag[x].depth)
        newGOs.append(GO)
    newGOs=[x for x in newGOs if x.startswith("biological_process")]
    #filter newGOs to remove elements with level and depth less than l and d
    if l!=0:
        newGOs=[x for x in newGOs if int(x.split(" ")[-2][1:])>=l]
    if d!=0:
        newGOs=[x for x in newGOs if int(x.split(" ")[-1][1:])>=d]
    #delete the ones with positive regulation or negative regulation
    newGOs=[x for x in newGOs if "regulation" not in x]
    GOs=Counter(newGOs)
    GOs_background=Counter(background)
    M = len(df)
    n = len(df_cut)
    p_values = {}
    enrichments = {}
    for go in GOs:
        N = GOs_background[go]
        x = GOs[go]
        enrichments[go] = (x/n)/(N/M)
        p_values[go] = hypergeom.sf(x-1, M, N, n)
    pvals = list(p_values.values())
    corrected_pvals_bonferroni = multipletests(pvals, method='bonferroni')[1]
    corrected_pvals_benjamini_hochberg = multipletests(pvals, method='fdr_bh')[1]
    data={'GO term':list(p_values.keys()),'p-value':list(p_values.values()),'corrected p-value Bonferroni':corrected_pvals_bonferroni,'corrected p-value Benjamini-Hochberg':corrected_pvals_benjamini_hochberg}
    dfGO = pd.DataFrame(data)
    dfGO['enrichment'] = dfGO['GO term'].map(enrichments)
    dfGO = dfGO.sort_values(by='p-value', ascending=True)
    return dfGO
GO_enrichments(0.05,refined=True,l=5,d=5)

,GO term,p-value,corrected p-value Bonferroni,corrected p-value Benjamini-Hochberg,enrichment
27,biological_process - pyrimidine-containing com...,0.000008,0.001412,0.001412,4.534873
76,biological_process - pyrimidine nucleoside mon...,0.000017,0.002948,0.001474,7.671493
93,biological_process - pyrimidine nucleoside met...,0.000039,0.006671,0.002224,10.959276
79,biological_process - pyrimidine ribonucleotide...,0.000191,0.032207,0.008052,6.575566
168,biological_process - protein-containing comple...,0.000255,0.043162,0.008632,10.959276
...,...,...,...,...,...
120,biological_process - purine nucleotide metabol...,0.989829,1.000000,0.998935,0.369414
138,biological_process - monocarboxylic acid metab...,0.991890,1.000000,0.998935,0.214888
51,biological_process - alpha-amino acid biosynth...,0.993612,1.000000,0.998935,0.204846
143,biological_process - nucleoside triphosphate m...,0.997773,1.000000,0.998935,0.169911
